# Project 1 - Group 6
###### <font color='gray'>Authors: Aisha Baitemoriva-Othman, Adonis McQueen, Angela Kim, Deja Prade, James Kang</font>

In [1]:
## Import libraries ##
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [4]:
## Import datasets ##

# Aisha's dataset: Rotten Tomatoes
df_rt_movie_info = pd.read_csv('data/rt.movie_info.tsv.gz',sep='\t')

# Adonis's dataset: Box Office Mojo
df1 = pd.read_csv("data/bom.movie_gross.csv.gz")

# Angela's dataset: The Numbers
df = pd.read_csv('data/tn.movie_budgets.csv.gz')

# Deja's dataset: TMDB
movie_type = pd.read_csv('data/tmdb.movies.csv.gz')

# James's datasets: IMDB
title_ratings = pd.read_csv('data/imdb.title.ratings.csv.gz')
title_basics = pd.read_csv('data/imdb.title.basics.csv.gz')
title_akas = pd.read_csv('data/imdb.title.akas.csv.gz')

### <font color='black'>Questions</font> 
<ul>
<li>*What are the highest gross values for both domestic and worldwide?*
<br>
<li>*What are the domestic and worldwide profits?*
<br>
<li>*Are there any values we can reasonably say won't help answer our questions, such as gross values that are too low and can be removed from our working dataset?*
</ul>